In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1, stop_words=["english"], token_pattern=r"[^\W\d_]+")
print (vectorizer)



CountVectorizer(stop_words=['english'], token_pattern='[^\\W\\d_]+')


In [16]:
with open("./data/alt.atheism.txt", "r", encoding='ISO-8859-1') as file:
    lines = file.readlines()


In [17]:
X_train = vectorizer.fit_transform(lines)
# число сообщений, длина словаря
num_samples, num_features = X_train.shape
print ('ns= %d, nf= %d' % (num_samples, num_features))

ns= 120487, nf= 14049


In [18]:
print (vectorizer.get_feature_names_out())

['a' 'aa' 'aaa' ... 'zvonko' 'zyklon' 'ÿ']
